In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI：跟踪本地训练模型的参数和指标

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-locally-trained-models.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-locally-trained-models.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ml_metadata/sdk-metric-parameter-tracking-for-locally-trained-models.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
     </a>
     </td>
</table>

## 概述

本笔记本演示了如何使用Vertex AI SDK for Python跟踪ML训练作业的指标和参数，并分析这些元数据。

了解有关[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)的更多信息。

### 目标

在这本笔记本中，您将学习如何使用`Vertex ML Metadata`来跟踪训练参数和评估指标。

本教程使用以下谷歌云ML服务：

- `Vertex ML Metadata`
- `Vertex AI Experiments`

执行的步骤包括：

- 跟踪本地训练模型的参数和指标。
- 提取并分析实验中的所有参数和指标。

数据集

在这份笔记本中，我们将训练一个简单的分布式神经网络（DNN）模型，根据[auto-mpg数据集](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg)中的汽车信息来预测汽车的每加仑英里数（MPG）。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您预期的使用量生成成本估算。

## 安装

安装执行此笔记本所需的软件包。

In [ ]:
! pip install --upgrade --quiet google-cloud-aiplatform \
                                tensorflow==2.11

### 仅限Colab：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置你的项目ID

**如果你不知道你的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`。
- 运行 `gcloud project list`。
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project $PROJECT_ID

区域

您还可以更改由Vertex AI使用的`REGION`变量。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐号
根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI工作台**
- 不需要任何操作，因为您已经验证通过。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
- 请参考如何为您的服务账户授予云存储权限，网址为 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples

导入库并定义常量

导入必需的库。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from google.cloud import aiplatform
from tensorflow.python.keras import Sequential, layers
from tensorflow.python.keras.utils import data_utils

定义一些常量

In [ ]:
EXPERIMENT_NAME = "my-experiment-name-unique"  # @param {type:"string"}

## 概念

为了更好地理解参数和指标是如何存储和组织的，我们想要介绍以下概念：

### 实验
实验描述了一个将您的运行和创建的工件组合成一个逻辑会话的上下文。例如，在这个笔记本中，您创建一个实验并将数据记录到该实验中。

### 运行
一次运行代表你在执行实验时所进行的单一路径/途径。一次运行包括您用作输入或输出的工件，以及您在这个执行中使用的参数。一个实验可以包含多次运行。

开始跟踪参数和度量指标

您可以使用Python的Vertex SDK来跟踪本地训练模型的度量和参数。

在下面的例子中，您将训练一个简单的分布式神经网络（DNN）模型，根据[auto-mpg数据集](https://www.kaggle.com/devanshbesain/exploration-and-analysis-auto-mpg)中的汽车信息来预测汽车的每加仑英里数（MPG）。

### 载入并处理训练数据集

下载并处理数据集。

In [ ]:
def read_data(uri):
    dataset_path = data_utils.get_file("auto-mpg.data", uri)
    column_names = [
        "MPG",
        "Cylinders",
        "Displacement",
        "Horsepower",
        "Weight",
        "Acceleration",
        "Model Year",
        "Origin",
    ]
    raw_dataset = pd.read_csv(
        dataset_path,
        names=column_names,
        na_values="?",
        comment="\t",
        sep=" ",
        skipinitialspace=True,
    )
    dataset = raw_dataset.dropna()
    dataset["Origin"] = dataset["Origin"].map(
        lambda x: {1: "USA", 2: "Europe", 3: "Japan"}.get(x)
    )
    dataset = pd.get_dummies(dataset, prefix="", prefix_sep="")
    return dataset


dataset = read_data(
    "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
)

拆分数据集用于训练和测试。

In [ ]:
def train_test_split(dataset, split_frac=0.8, random_state=0):
    train_dataset = dataset.sample(frac=split_frac, random_state=random_state)
    test_dataset = dataset.drop(train_dataset.index)
    train_labels = train_dataset.pop("MPG")
    test_labels = test_dataset.pop("MPG")

    return train_dataset, test_dataset, train_labels, test_labels


train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset)

对数据集中的特征进行归一化处理以获得更好的模型性能。

In [ ]:
def normalize_dataset(train_dataset, test_dataset):
    train_stats = train_dataset.describe()
    train_stats = train_stats.transpose()

    def norm(x):
        return (x - train_stats["mean"]) / train_stats["std"]

    normed_train_data = norm(train_dataset)
    normed_test_data = norm(test_dataset)

    return normed_train_data, normed_test_data


normed_train_data, normed_test_data = normalize_dataset(train_dataset, test_dataset)

### 定义机器学习模型和训练功能

In [ ]:
def train(
    train_data,
    train_labels,
    num_units=64,
    activation="relu",
    dropout_rate=0.0,
    validation_split=0.2,
    epochs=1000,
):

    model = Sequential(
        [
            layers.Dense(
                num_units,
                activation=activation,
                input_shape=[len(train_dataset.keys())],
            ),
            layers.Dropout(rate=dropout_rate),
            layers.Dense(num_units, activation=activation),
            layers.Dense(1),
        ]
    )

    model.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])
    print(model.summary())

    history = model.fit(
        train_data, train_labels, epochs=epochs, validation_split=validation_split
    )

    return model, history

### 初始化用于 Python 的 Vertex AI SDK 并创建一个实验

对 Vertex AI 进行*client*的初始化，并创建一个实验。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, experiment=EXPERIMENT_NAME)

开始几次模型训练运行

每次运行的训练参数和指标都有记录。

In [ ]:
parameters = [
    {"num_units": 16, "epochs": 3, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 16, "epochs": 10, "dropout_rate": 0.2},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.1},
    {"num_units": 32, "epochs": 10, "dropout_rate": 0.2},
]

for i, params in enumerate(parameters):
    aiplatform.start_run(run=f"auto-mpg-local-run-{i}")
    aiplatform.log_params(params)
    model, history = train(
        normed_train_data,
        train_labels,
        num_units=params["num_units"],
        activation="relu",
        epochs=params["epochs"],
        dropout_rate=params["dropout_rate"],
    )

    for metric, values in history.history.items():
        try:
            aiplatform.log_metrics({metric: values[-1]})
        except:
            aiplatform.log_metrics({metric: 0.0})

    loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
    try:
        aiplatform.log_metrics({"eval_loss": loss, "eval_mae": mae, "eval_mse": mse})
    except:
        aiplatform.log_metrics({"eval_loss": 0.0, "eval_mae": 0.0, "eval_mse": 0.0})

提取参数和指标到数据框中进行分析。

我们还可以将与任何实验相关的所有参数和指标提取到数据框中，以便进一步分析。

In [ ]:
experiment_df = aiplatform.get_experiment_df()
experiment_df

### 可视化实验的参数和度量标准

In [ ]:
plt.rcParams["figure.figsize"] = [15, 5]

ax = pd.plotting.parallel_coordinates(
    experiment_df.reset_index(level=0),
    "run_name",
    cols=[
        "param.num_units",
        "param.dropout_rate",
        "param.epochs",
        "metric.loss",
        "metric.val_loss",
        "metric.eval_loss",
    ],
    color=["blue", "green", "pink", "red"],
)
ax.set_yscale("symlog")
ax.legend(bbox_to_anchor=(1.0, 0.5))

在云控制台中可视化实验

运行以下命令来获取您项目的Vertex AI实验的URL。

In [ ]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/ai/platform/experiments/experiments?folder=&organizationId=&project={PROJECT_ID}"
)

清理

要清理本项目中使用的所有Google Cloud资源，您可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的单个资源：

- 实验（可以在GCP控制台UI中手动删除）